In [1]:
import os
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline


import glob
from astropy.io import fits
from astropy.stats import sigma_clip
from scipy.ndimage import interpolation as interp

#from skimage.feature.register_translation import (register_translation, _upsampled_dft)

# import aotools
import astroscrappy
from scipy.optimize import leastsq
# import ccdproc
import math

In [2]:
from scipy.ndimage.filters import median_filter, gaussian_filter


def guess_gaussian_parameters(d):
    """
    This image guesses the maximum intensity of the 
    image by obtaining a smoothed version of the image 
    via median + gaussian filtering. Then, finds the 
    maximum of the smoothed image. Using the median-filtered 
    image, the algorithm also estimates the width of the PSF and 
    with this value and an estimate of the volume, the amplitude of 
    such gaussian.

    Input

    d       Numpy array that contains the values of the pixels.

    Output

    x0      x coordinate of the image maximum
    
    y0      y coordinate of the image maximum

    sigma   Width of the PSF

    A       Estimated amplitude of the gaussian function
    """

    # First, smooth the image with a median filter. For this, find 
    # the optimal window as the square-root of the geometric mean 
    # between the sizes of the array. This step is good to kill outliers:
    window = int(np.sqrt(np.sqrt(d.shape[0] * d.shape[1])))
    if window % 2 == 0:
        window = window + 1

        # Originally there was a median filter here, but it really adds a lot of time to the routine
    #d_mfiltered = median_filter(d,size = window)
    # Going to try without it to see if it makes a significant difference for vetoing
    d_gfiltered = gaussian_filter(d, sigma=window)

    # Now, find the maximum of this image:
    y0, x0 = np.where(d_gfiltered == np.max(d_gfiltered))

    # Take the first element. This helps in two cases: (1) only one maximum has 
    # been found, the outputs are numpy arrays and you want to extract the numbers 
    # and (2) in case there are several maximums (this has not happened so 
    # far but could in principle), pick the first of them:
    y0 = y0[0]
    x0 = x0[0]

    # Now estimate the width of the PSF by taking a "cross-plot" using the 
    # maximum values found:
    x_cut = d[:, int(x0)]
    sigma_x = (np.sum(x_cut * (np.abs(np.arange(len(x_cut)) - y0))) / np.sum(x_cut)) / 3.
    y_cut = d[int(y0), :]
    sigma_y = (np.sum(y_cut * (np.abs(np.arange(len(y_cut)) - x0))) / np.sum(y_cut)) / 3.
    sigma = np.sqrt(sigma_x * sigma_y)

    # (Under) estimate amplitude assuming a gaussian function:
    A = (np.sum(d - np.median(d)) / (2. * np.pi * sigma ** 2))

    return x0, y0, sigma, 2. * A


/tmp/ipykernel_17333/3376877513.py:1: DeprecationWarning: Please use `median_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import median_filter, gaussian_filter
/tmp/ipykernel_17333/3376877513.py:1: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import median_filter, gaussian_filter


In [3]:
datadir = '/home/tehan/PycharmProjects/Shane-AO-Reduction/Raw_Data/data-2023-04-09-AO-Paul.Robertson/'
os.chdir(datadir)

#These are the id numbers of the fitz files
firstID = 1;
lastID = 9999;

##Let's go through our images and remove the bad frames

all_object_list = []

veto_list = []

#Let's just make a list of every object
for ID in range(firstID, lastID + 1):
    try:
        if ID < 100:
            if ('s00' + str(ID) + '.fits'):
                header = fits.getheader('s00' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s00' + str(ID) + '.fits')


        elif 99 < ID < 1000:
            if ('s0' + str(ID) + '.fits'):
                header = fits.getheader('s0' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s0' + str(ID) + '.fits')

        elif 999 < ID < 10000:
            if ('s' + str(ID) + '.fits'):
                header = fits.getheader('s' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s' + str(ID) + '.fits')

    except OSError:
        continue
    except KeyError:
        continue

flat_obj_list = []
#let's remove the darks
for object in all_object_list:
    header = fits.getheader(object)
    obj = header['OBJECT']
    if obj == 'dark':
        continue
    else:
        flat_obj_list.append(object)




In [4]:
#Now we go through each object here, and we make a list of all the bad ones

veto_list = []

for obj in flat_obj_list:
    header = fits.getheader(obj)
    try:
        image_data = fits.getdata(obj)
        center=(1100,730)
        image_window = image_data[(center[1] - 200):(center[1] + 200), (center[0] - 200):(center[0] + 200)]  # we use the window we care about
        # shrink the window to avoid companion

        print(obj)
        print(header['OBJECT'])
        if 'flat' in header['OBJECT'].lower():
            cent_counts = np.mean(image_window)
            print('Flat!')
        else:

            x0, y0, sigma, A = guess_gaussian_parameters(
                image_window)  #need to do this for the window, not the whole image

            print(image_window[y0,x0])
            #now that we have the centroid, let's make a window about the centroid, and take the average
            centroid_window = image_window[y0 - 1:y0 + 1, x0 - 1:x0 + 1]
            cent_counts = np.mean(centroid_window)  #let's try max instead of mean

        print(cent_counts)

        if cent_counts > 25000:  #Setting our max counts at 25000 here
            veto_list.append(obj)
            print('Appended!')
        elif cent_counts < 500:  #Setting our minimum counts at 6500 here
            veto_list.append(obj)
            print('Appended!')
        elif math.isnan(cent_counts) == True:
            veto_list.append(obj)
            print('Appended!')
        else:
            continue
    except ValueError:
        continue


#Just median filter: ~94 s, appears accurate
#Just Gaussian filter: ~ 1 s, appears accurate



s0370.fits
Sky flat
Flat!
7.6262374
Appended!
s0371.fits
Sky flat
Flat!
7111.952
s0372.fits
Sky flat
Flat!
7019.3975
s0373.fits
Sky flat
Flat!
13602.839
s0374.fits
Sky flat
Flat!
13739.582
s0375.fits
Sky flat
Flat!
13774.679
s0376.fits
Sky flat
Flat!
13821.082
s0377.fits
Sky flat
Flat!
13992.814
s0378.fits
Sky flat
Flat!
14715.875
s0379.fits
Sky flat
Flat!
14391.814
s0380.fits
Sky flat
Flat!
13928.421
s0381.fits
Sky flat
Flat!
13773.166
s0382.fits
Sky flat
Flat!
13614.993
s0383.fits
Sky flat
Flat!
13442.11
s0384.fits
Sky flat
Flat!
11555.669
s0385.fits
Sky flat
Flat!
16703.27
s0386.fits
Sky flat
Flat!
16433.793
s0387.fits
Sky flat
Flat!
18219.266
s0388.fits
Sky flat
Flat!
21184.422
s0389.fits
Sky flat
Flat!
16196.672
s0390.fits
Sky flat
Flat!
16486.502
s0391.fits
Sky flat
Flat!
16483.91
s0392.fits
Sky flat
Flat!
16898.84
s0393.fits
Sky flat
Flat!
17747.328
s0394.fits
Sky flat
Flat!
18003.008
s0395.fits
Sky flat
Flat!
18121.842
s0396.fits
Sky flat
Flat!
18455.412
s0397.fits
Sky flat
Fla

/home/tehan/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/tehan/.local/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


23776.969
22291.5
s0511.fits
TIC-362025009
13523.125
13051.359
s0512.fits
TIC-362025009
20145.594
19501.266
s0513.fits
TIC-362025009
1321.5312
1250.5312
s0514.fits
TIC-362025009
1288.6875
1257.2344
s0515.fits
TIC-362025009
1244.7812
1191.2188
s0516.fits
TIC-362025009
1213.625
1173.3906
s0517.fits
TIC-362025009
1143.3125
1127.6562
s0518.fits
TIC-362025009
1216.7188
1163.2344
s0519.fits
TIC-362025009
1199.2812
1203.0
s0520.fits
TIC-362025009
1196.6875
1162.5938
s0521.fits
TIC-362025009
1245.0
1200.8125
s0522.fits
TIC-362025009
1186.375
1162.9531
s0523.fits
TIC-362025009
21140.031
19235.328
s0524.fits
TIC-362025009
18548.188
16450.672
s0525.fits
TIC-362025009
20096.875
17730.11
s0526.fits
TIC-362025009
20912.031
18534.516
s0527.fits
TIC-362025009
15724.9375
14348.453
s0528.fits
TIC-362025009
13663.219
13153.797
s0529.fits
TIC-362025009
16217.3125
14692.656
s0530.fits
TIC-362025009
23351.469
21792.25
s0531.fits
TIC-362025009
19612.625
18020.844
s0532.fits
TIC-362025009
12997.469
11887.875


In [5]:
#Write it to a text file for later
with open('veto_list.txt', 'w') as f:
    for item in veto_list:
        f.write("%s\n" % item)